In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load dataset pengguna (users) dan kafe (cafes)
df_users = pd.read_csv("path_to_users_dataset.csv")
df_cafes = pd.read_csv("path_to_cafes_dataset.csv")

# Menggabungkan kolom-kolom minat pengguna menjadi satu kolom teks
df_users["user_interests"] = df_users[["Kpopers", "JapanLovers", "AnimalLovers", "Quite", "MusicLovers",
                                       "BookLovers", "ArtLovers", "ViewsLovers", "CoffeeLovers", "NonCoffeeLovers"]].apply(lambda x: ' '.join(map(str, x)), axis=1)

# Menggabungkan kolom-kolom fitur kafe menjadi satu kolom teks
df_cafes["cafe_features"] = df_cafes[["MeetingCafe", "StudyCafe", "Cafe with a good views", "FamilyCafe",
                                     "CountryFood", "Smoking", "NonSmoking", "Coffee", "NonCoffee", "GardenCafe"]].apply(lambda x: ' '.join(map(str, x)), axis=1)

# Menggunakan TF-IDF Vectorizer untuk mengonversi teks ke vektor
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
user_interests_matrix = tfidf_vectorizer.fit_transform(df_users["user_interests"])
cafe_features_matrix = tfidf_vectorizer.transform(df_cafes["cafe_features"])

# Menghitung similarity antara minat pengguna dan fitur kafe menggunakan cosine similarity
cosine_similarities = linear_kernel(user_interests_matrix, cafe_features_matrix)

# Fungsi untuk mendapatkan rekomendasi kafe berdasarkan minat pengguna
def get_cafe_recommendations(user_id, cosine_similarities=cosine_similarities):
    user_idx = df_users[df_users['User_id'] == user_id].index[0]
    cafe_scores = list(enumerate(cosine_similarities[user_idx]))
    cafe_scores = sorted(cafe_scores, key=lambda x: x[1], reverse=True)
    cafe_indices = [score[0] for score in cafe_scores]
    
    # Mengembalikan nama kafe yang direkomendasikan
    recommended_cafes = df_cafes.loc[cafe_indices]["Name"].tolist()
    return recommended_cafes

# Contoh penggunaan: Mendapatkan rekomendasi kafe untuk User_id 0
user_id = 0
recommendations = get_cafe_recommendations(user_id)
print(f"Rekomendasi kafe untuk User_id {user_id}:", recommendations)
